## example-format-1

### ast 
- 一个合法的 ast 节点数据结构，应该是可以转换成字符串代码的

In [17]:
import (
    "go/ast"
    "go/token"
)

- 构建一个函数声明的 AST 节点

In [56]:
fn := &ast.FuncDecl{
        Name: ast.NewIdent("myfunc"), //函数名称
        Type: &ast.FuncType{ //这里定义函数参数和返回结果
            Params: &ast.FieldList{ //函数参数
            },
            Results: &ast.FieldList{
                List: []*ast.Field{
                    &ast.Field{
                        Type: &ast.StarExpr{ //返回指针类型
                            X: ast.NewIdent("ptr"),
                            //.e.g *Endpoints
                        },
                    },
                    &ast.Field{
                        Type: &ast.SelectorExpr{
                            X:   ast.NewIdent("a"),
                            Sel: ast.NewIdent("b"),
                        },
                        //返回包对象.e.g : a.b
                    },
                },
            },
        },
        Body: nil, //函数内语句包含在这里
    }

- 打印 AST 节点信息

In [57]:
fset := token.NewFileSet()
ast.Print(fset, fn)

     0  *ast.FuncDecl {
     1  .  Name: *ast.Ident {
     2  .  .  NamePos: -
     3  .  .  Name: "myfunc"
     4  .  }
     5  .  Type: *ast.FuncType {
     6  .  .  Func: -
     7  .  .  Params: *ast.FieldList {
     8  .  .  .  Opening: -
     9  .  .  .  Closing: -
    10  .  .  }
    11  .  .  Results: *ast.FieldList {
    12  .  .  .  Opening: -
    13  .  .  .  List: []*ast.Field (len = 2) {
    14  .  .  .  .  0: *ast.Field {
    15  .  .  .  .  .  Type: *ast.StarExpr {
    16  .  .  .  .  .  .  Star: -
    17  .  .  .  .  .  .  X: *ast.Ident {
    18  .  .  .  .  .  .  .  NamePos: -
    19  .  .  .  .  .  .  .  Name: "ptr"
    20  .  .  .  .  .  .  }
    21  .  .  .  .  .  }
    22  .  .  .  .  }
    23  .  .  .  .  1: *ast.Field {
    24  .  .  .  .  .  Type: *ast.SelectorExpr {
    25  .  .  .  .  .  .  X: *ast.Ident {
    26  .  .  .  .  .  .  .  NamePos: -
    27  .  .  .  .  .  .  .  Name: "a"
    28  .  .  .  .  .  .  }
    29  .  .  .  .  .  .  Sel: *ast.Ident {
    30

- 把 AST 节点转换成 go 源码

In [67]:
import "bytes"
import "go/format"
import "fmt"

In [68]:
var buf bytes.Buffer

err := format.Node(&buf, fset, fn)
if err != nil {
    panic(err)
}
fmt.Println(buf.String())

func myfunc() (*ptr, a.b)


26 <nil>

- 补齐一下代码

In [60]:
buf.WriteString("{a:=1}")

6 <nil>

In [61]:
fmt.Println(buf.String())

func myfunc() (*ptr, a.b){a:=1}


32 <nil>

In [62]:
import (
    "go/ast"
    "go/parser"
    "go/token"
)

- parser.ParseFile 的调用有报错
- 似乎是针对完整的 go 源码文件的，也就是需要包括 package 等完整的声明

In [69]:
fset := token.NewFileSet()

file, err := parser.ParseFile(fset, "", buf.String(), 0)
if err != nil {
    panic(err)
}

ERROR: 1:1: expected 'package', found 'func'

In [70]:
ast.Print(fset, file)

     0  *ast.File {
     1  .  Package: -
     2  .  Name: *ast.Ident {
     3  .  .  NamePos: -
     4  .  .  Name: ""
     5  .  }
     6  .  Scope: *ast.Scope {
     7  .  .  Objects: map[string]*ast.Object (len = 0) {}
     8  .  }
     9  }


- 补齐代码

In [71]:
var src bytes.Buffer
src.WriteString("package main\n")
src.WriteString(buf.String())
fmt.Println(src.String())

package main
func myfunc() (*ptr, a.b)


39 <nil>

In [72]:
fset := token.NewFileSet()

file, err := parser.ParseFile(fset, "", src.String(), 0)
if err != nil {
    panic(err)
}
ast.Print(fset, file)

     0  *ast.File {
     1  .  Package: 1:1
     2  .  Name: *ast.Ident {
     3  .  .  NamePos: 1:9
     4  .  .  Name: "main"
     5  .  }
     6  .  Decls: []ast.Decl (len = 1) {
     7  .  .  0: *ast.FuncDecl {
     8  .  .  .  Name: *ast.Ident {
     9  .  .  .  .  NamePos: 2:6
    10  .  .  .  .  Name: "myfunc"
    11  .  .  .  .  Obj: *ast.Object {
    12  .  .  .  .  .  Kind: func
    13  .  .  .  .  .  Name: "myfunc"
    14  .  .  .  .  .  Decl: *(obj @ 7)
    15  .  .  .  .  }
    16  .  .  .  }
    17  .  .  .  Type: *ast.FuncType {
    18  .  .  .  .  Func: 2:1
    19  .  .  .  .  Params: *ast.FieldList {
    20  .  .  .  .  .  Opening: 2:12
    21  .  .  .  .  .  Closing: 2:13
    22  .  .  .  .  }
    23  .  .  .  .  Results: *ast.FieldList {
    24  .  .  .  .  .  Opening: 2:15
    25  .  .  .  .  .  List: []*ast.Field (len = 2) {
    26  .  .  .  .  .  .  0: *ast.Field {
    27  .  .  .  .  .  .  .  Type: *ast.StarExpr {
    28  .  .  .  .  .  .  .  .  Star: 2:16
    29